In [47]:
import os
import pandas as pd
from tqdm import tqdm
from re import sub
import numpy as np
from thefuzz import fuzz
import shortuuid
import xml.etree.ElementTree as ET
import re
import sys
from gensim.utils import simple_preprocess
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import SparseTermSimilarityMatrix, WordEmbeddingSimilarityIndex, SoftCosineSimilarity

In [48]:
DATASETS_FOLDER = "datasets"    #the folder that contains the dataset directories to read in
FORMATTED_DATA_FILENAME = "true_data.json"  #the name of the file that contains the data to read in
DATASETS = ["DMoP", "MSRP", "ETPC"]     #the folders in the DATASETS_FOLDER should be named like the datasets here
OUT_DIR = "output"      #the directory to output the formatted json in

FUZZY = "fuzzy_based_result"
SEMANTIC = "semantic_based_result"

pd.set_option("display.max_colwidth", None)

In [49]:
df = pd.read_json(os.path.join(OUT_DIR, FORMATTED_DATA_FILENAME), orient = "index")
df.head()

,dataset,id_1,id_2,text_1,text_2,is_paraphrase
0,DMoP,HiDc8dkT,3n3Na3BD,Roy of the Rovers,Roy of the Rovers,True
1,DMoP,5KfcxDBb,QUzQei9u,"Roy of the Rovers is a British comic strip about the life and times of a fictional footballer named Roy Race, who played for Melchester Rovers. The strip first appeared in the ""Tiger"" in 1954, before giving its name to a weekly (and later monthly) comic magazine, published by IPC and Fleetway from 1976 until 1995, in which it was the main feature.","Roy of the Rovers is a British funny cartoon about the life and times of an anecdotal footballer named Roy Race, who played for Melchester Rovers. The strip originally showed up in the ""Tiger"" in 1954, preceding giving its name to a week after week (and later month to month) comic magazine, distributed by IPC and Fleetway from 1976 until 1995, in which it was the principle include.",True
2,DMoP,gxShcSRt,5nSAkBfd,"The weekly strip ran until 1993, following Roy's playing career until its conclusion after he lost his left foot in a helicopter crash. When the monthly comic was launched later that year the focus switched to Roy's son Rocky, who also played for Melchester. This publication was short-lived, and folded after only 19 issues. The adventures of the Race family were subsequently featured in the monthly ""Match of the Day"" football magazine, in which father and son were reunited as manager and player respectively. These strips began in 1997 and continued until the magazine's closure in May 2001.","The week after week strip kept running until 1993, after Roy's playing vocation until its decision after he lost his left foot in a helicopter crash. At the point when the month to month comic was propelled soon thereafter the center changed to Roy's child Rocky, who additionally played for Melchester. This production was brief, and collapsed after just 19 issues. The undertakings of the Race family were in this manner highlighted in the month to month ""Match of the Day"" football magazine, in which father and child were brought together as chief and player separately. These strips started in 1997 and proceeded until the magazine's conclusion in May 2001.",True
3,DMoP,ERwDq4GC,mFG5GR3n,"Football-themed stories were a staple of British comics for boys from the 1950s onwards, and Roy of the Rovers was the most popular. To keep the strip exciting, Melchester was almost every year either competing for major honours or struggling against relegation to a lower division; a normal, uneventful season of mid-table mediocrity was unknown at Melchester Rovers. The strip followed the structure of the actual English football season, thus there were several months each year in summer when there was no league football. By far the most common summer storyline saw Melchester touring a fictional country in an exotic part of the world, often South America, where they would invariably be kidnapped and held to ransom. The average reader probably stayed with the comic regularly for only three or four years, therefore storylines were sometimes recycled; during the first ten years of his playing career, Roy was kidnapped at least four times. Roy also made numerous appearances for England, depicted playing alongside actual players such as Malcolm Macdonald and Trevor Francis.","Football-themed stories were a staple of British funnies for young men from the 1950s onwards, and Roy of the Rovers was the most prevalent. To keep the strip energizing, Melchester was pretty much consistently either seeking real distinctions or battling against transfer to a lower division; a typical, uneventful period of mid-table unremarkableness was obscure at Melchester Rovers. The strip pursued the structure of the real English football season, along these lines there were a while every year in summer when there was no association football. By a long shot the most well-known summer storyline saw Melchester visiting an anecdotal nation in a fascinating piece of the world, regularly South 

In [50]:
#Check for paraphrase with fuzzy based
fuzzy_results = []
print("Checking for paraprhases with the fuzzy-based method. Dataframe rows to process: " + str(len(df)))
for i, row in tqdm(df.iterrows()):
    fuzzy_results.append(fuzz.ratio(row["text_1"], row["text_2"]))

df[FUZZY] = fuzzy_results

Checking for paraprhases with the fuzzy-based method. Dataframe rows to process: 11972


11972it [00:01, 9699.60it/s] 


In [52]:
stopwords = ['the', 'and', 'are', 'a']

In [53]:
#Semantic based
def preprocess(doc):
        # Tokenize, clean up input document string
        doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
        doc = sub(r'<[^<>]+(>|$)', " ", doc)
        doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
        doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
        return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]


In [72]:
# Preprocess the documents, including the query string
def check_semantic(string_1, string_2):
    # Preprocess the documents, including the query string
    corpus = [preprocess(string_1)]
    query = preprocess(string_2)

    # use a pre trained model: https://huggingface.co/fse/glove-wiki-gigaword-50 , https://nlp.stanford.edu/pubs/glove.pdf
    glove = api.load("glove-wiki-gigaword-50")
    similarity_index = WordEmbeddingSimilarityIndex(glove)

    # Build the term dictionary, TF-idf model
    dictionary = Dictionary(corpus + [query])
    #print(dictionary)
    tfidf = TfidfModel(dictionary=dictionary)

    # Create the term similarity matrix.
    similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

    query_tf = tfidf[dictionary.doc2bow(query)]

    #print(dictionary.doc2bow(query))

    index = SoftCosineSimilarity(
        tfidf[[dictionary.doc2bow(document) for document in corpus]],
        similarity_matrix)

    return index[query_tf]

In [74]:
semantic_results = []

for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    if i >= 50:
        semantic_results.append(-1)
    else:
        semantic_results.append(float(check_semantic(row["text_1"], row["text_2"])))

df[SEMANTIC] = semantic_results

  0%|          | 0/11972 [00:00<?, ?it/s]

In [ ]:
df.head(10)

,dataset,id_1,id_2,text_1,text_2,is_paraphrase,fuzzy_based_result,semantic_based_result
0,DMoP,HiDc8dkT,3n3Na3BD,Roy of the Rovers,Roy of the Rovers,True,100,0.0
1,DMoP,5KfcxDBb,QUzQei9u,"Roy of the Rovers is a British comic strip about the life and times of a fictional footballer named Roy Race, who played for Melchester Rovers. The strip first appeared in the ""Tiger"" in 1954, before giving its name to a weekly (and later monthly) comic magazine, published by IPC and Fleetway from 1976 until 1995, in which it was the main feature.","Roy of the Rovers is a British funny cartoon about the life and times of an anecdotal footballer named Roy Race, who played for Melchester Rovers. The strip originally showed up in the ""Tiger"" in 1954, preceding giving its name to a week after week (and later month to month) comic magazine, distributed by IPC and Fleetway from 1976 until 1995, in which it was the principle include.",True,83,[0.4854329]
2,DMoP,gxShcSRt,5nSAkBfd,"The weekly strip ran until 1993, following Roy's playing career until its conclusion after he lost his left foot in a helicopter crash. When the monthly comic was launched later that year the focus switched to Roy's son Rocky, who also played for Melchester. This publication was short-lived, and folded after only 19 issues. The adventures of the Race family were subsequently featured in the monthly ""Match of the Day"" football magazine, in which father and son were reunited as manager and player respectively. These strips began in 1997 and continued until the magazine's closure in May 2001.","The week after week strip kept running until 1993, after Roy's playing vocation until its decision after he lost his left foot in a helicopter crash. At the point when the month to month comic was propelled soon thereafter the center changed to Roy's child Rocky, who additionally played for Melchester. This production was brief, and collapsed after just 19 issues. The undertakings of the Race family were in this manner highlighted in the month to month ""Match of the Day"" football magazine, in which father and child were brought together as chief and player separately. These strips started in 1997 and proceeded until the magazine's conclusion in May 2001.",True,75,[0.6327045]
3,DMoP,ERwDq4GC,mFG5GR3n,"Football-themed stories were a staple of British comics for boys from the 1950s onwards, and Roy of the Rovers was the most popular. To keep the strip exciting, Melchester was almost every year either competing for major honours or struggling against relegation to a lower division; a normal, uneventful season of mid-table mediocrity was unknown at Melchester Rovers. The strip followed the structure of the actual English football season, thus there were several months each year in summer when there was no league football. By far the most common summer storyline saw Melchester touring a fictional country in an exotic part of the world, often South America, where they would invariably be kidnapped and held to ransom. The average reader probably stayed with the comic regularly for only three or four years, therefore storylines were sometimes recycled; during the first ten years of his playing career, Roy was kidnapped at least four times. Roy also made numerous appearances for England, depicted playing alongside actual players such as Malcolm Macdonald and Trevor Francis.","Football-themed stories were a staple of British funnies for young men from the 1950s onwards, and Roy of the Rovers was the most prevalent. To keep the strip energizing, Melchester was pretty much consistently either seeking real distinctions or battling against transfer to a lower division; a typical, uneventful period of mid-table unremarkableness was obscure at Melchester Rovers. The strip pursued the structure of the real English football season, along these lines there were a while every year in summer when there was no association football. By a long shot the most well-known summer storyline saw Melchester vis